In [8]:
from sfm.data.sci_data.SFMDecTokenizer import SFMDecTokenizer
import torch
import os

In [9]:
tokenizer_home = '/hai1/ds_dataset/llama2/llama-2-7b'
tokenizer = SFMDecTokenizer.from_pretrained(
    tokenizer_home,
    prot_spm_path='/blob/shufxi/data/scigpt/ur50bpe/bpe',
    dna_spm_path='/blob/shufxi/data/scigpt/dnabpe/bpe'
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'SFMDecTokenizer'.


You are using the default legacy behaviour of the <class 'sfm.data.sci_data.SFMDecTokenizer.SFMDecTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


[2024-01-13 14:01:14.425][INFO]: Loading protein sentencepiece model from /blob/shufxi/data/scigpt/ur50bpe/bpe.model and /blob/shufxi/data/scigpt/ur50bpe/bpe.vocab
[2024-01-13 14:01:15.336][INFO]: Loading DNA sentencepiece model from /blob/shufxi/data/scigpt/dnabpe/bpe.model and /blob/shufxi/data/scigpt/dnabpe/bpe.vocab
[2024-01-13 14:01:15.980][INFO]: Tokenizer has 38991 tokens


In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
ckpt_home = '/hai1/shufxi/scigpt/7bv2/stageB/finetune/solubility/1e-5/13/global_step968'

def show_ckpt(name, ckpt):
    for k, v in ckpt.items():
        if 'dummy' not in k:
            print(name, k, v.shape)

model = AutoModelForCausalLM.from_pretrained(tokenizer_home)

model_dict = model.state_dict()
ckpt_dict = {}
layer0 = torch.load(os.path.join(ckpt_home, "layer_00-model_states.pt"), map_location=torch.device("cpu"))
ckpt_dict['model.embed_tokens.weight'] = layer0['embed_tokens.weight']
show_ckpt('layer0', layer0)

for l in range(0, 32):
    l_index = str(l + 1).zfill(2)
    layer = torch.load(os.path.join(ckpt_home, f"layer_{l_index}-model_states.pt"), map_location=torch.device("cpu"))
    show_ckpt(l_index, layer)
    for k in layer:
        if "dummy" in k or 'rotary_emb' in k:
            continue
        ckpt_dict[f"model.layers.{l}.{k}"] = layer[k]
layer = torch.load(os.path.join(ckpt_home, "layer_33-model_states.pt"), map_location=torch.device("cpu"))
show_ckpt(33, layer)
ckpt_dict["model.norm.weight"] = layer["norm.weight"]

layer = torch.load(os.path.join(ckpt_home, "layer_34-model_states.pt"), map_location=torch.device("cpu"))
show_ckpt(33, layer)
ckpt_dict["lm_head.weight"] = layer["lm_head.weight"]
model_dict.update(ckpt_dict)

model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(model_dict)

model.cuda()

Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.77s/it]


layer0 embed_tokens.weight torch.Size([38991, 4096])
01 self_attn.q_proj.weight torch.Size([4096, 4096])
01 self_attn.k_proj.weight torch.Size([4096, 4096])
01 self_attn.v_proj.weight torch.Size([4096, 4096])
01 self_attn.o_proj.weight torch.Size([4096, 4096])
01 mlp.gate_proj.weight torch.Size([11008, 4096])
01 mlp.up_proj.weight torch.Size([11008, 4096])
01 mlp.down_proj.weight torch.Size([4096, 11008])
01 input_layernorm.weight torch.Size([4096])
01 post_attention_layernorm.weight torch.Size([4096])
02 self_attn.q_proj.weight torch.Size([4096, 4096])
02 self_attn.k_proj.weight torch.Size([4096, 4096])
02 self_attn.v_proj.weight torch.Size([4096, 4096])
02 self_attn.o_proj.weight torch.Size([4096, 4096])
02 mlp.gate_proj.weight torch.Size([11008, 4096])
02 mlp.up_proj.weight torch.Size([11008, 4096])
02 mlp.down_proj.weight torch.Size([4096, 11008])
02 input_layernorm.weight torch.Size([4096])
02 post_attention_layernorm.weight torch.Size([4096])
03 self_attn.q_proj.weight torch.Size

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(38991, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [11]:
from sfm.data.prot_data.dataset import DownstreamLMDBDataset
from sfm.data.sci_data.dataset import ProteinLmdbDataset

In [12]:
class ProtArgs(object):
    def __init__(self, task_name, data_basepath):
        self.task_name = task_name
        self.data_basepath = data_basepath
        self.max_length = 2048

args = ProtArgs(
    task_name = 'solubility',
    data_basepath='/pfm/data/bfm_benchmark'
)
dataset_dict = DownstreamLMDBDataset.load_dataset(args)

[2024-01-13 14:03:01.775][WARNING]: Removed 0 examples from the dataset because they are longer than 2048.
[2024-01-13 14:03:01.790][WARNING]: Removed 0 examples from the dataset because they are longer than 2048.
[2024-01-13 14:03:01.794][WARNING]: Removed 0 examples from the dataset because they are longer than 2048.


In [13]:
valid_dataset = ProteinLmdbDataset(
    task_name=args.task_name,
    lmdb_dataset=dataset_dict['valid'],
    lmdb_vocab=dataset_dict['valid'].vocab,
    tokenizer=tokenizer,
)

test_dataset = ProteinLmdbDataset(
    task_name=args.task_name,
    lmdb_dataset=dataset_dict['test'],
    lmdb_vocab=dataset_dict['test'].vocab,
    tokenizer=tokenizer,
)

[2024-01-13 14:03:01.803][INFO]: Loaded solubility with 6942 lines, type binary
[2024-01-13 14:03:01.804][INFO]: Loaded solubility with 1999 lines, type binary


In [14]:
dataset_dict['valid'][0]['target']

array([0])

In [15]:
from tqdm import tqdm

In [16]:
def parse_pred(task_type, pred):
    if task_type == 'binary':
        assert pred in ['true', 'false'], f'invalid pred {pred}'
        return 1 if pred == 'true' else 0
    elif task_type == 'regression':
        return float(pred)
    elif task_type == 'classification':
        return int(pred)
    else:
        pred = pred.split(',')
        return [int(p.strip()) for p in pred]

pred_list = []
ground_truth_list = []

for i in tqdm(range(len(test_dataset))):
    input_ids, labels = test_dataset[i]
    last_prompt_token_pos = 0
    for j in range(len(input_ids)):
        if labels[j] == -100:
            last_prompt_token_pos = j

    prompt_ids = input_ids[:last_prompt_token_pos+1]

    pred = model.generate(
        prompt_ids.unsqueeze(0).cuda(),
        num_beams=4,
        max_new_tokens=10,
        num_return_sequences=1,
    )[0]
    pred_label = pred[last_prompt_token_pos+1:]
    pred_label_text = tokenizer.decode(pred_label[:-1])
    task_type = DownstreamLMDBDataset.TASKINFO[args.task_name]["type"]
    pred_label_value = parse_pred(task_type, pred_label_text)

    pred_list.append(pred_label_value)
    ground_truth_list.append(dataset_dict['test'][i]['target'][0])


  0%|          | 0/1999 [00:00<?, ?it/s]

 30%|███       | 604/1999 [04:49<11:12,  2.07it/s]Bad pipe message: %s [b'M\xb4\xb7\xe4\xcd\xf5\xec\xfd\xd0lI\xed7\xac\xd8\x16\x12\xd8 y\x1d\xd8\xb3\x0fjp^O\xdbbb5\xf6(($\xe4\xe2Wn\xf5\xfd\xc0\xd8\x9e7C\x00\x03fE\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07', b'\x08\t\x08\n\x08\x0b\x08']
Bad pipe message: %s [b'\x05\x08\x06']
Bad pipe message: %s [b'\x05\x01\x06', b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \x96\x0b\xdb\xa0\xa8\xf8\x87j\xc3t\x06\x19\x86\xe6_\x88\xe1\xc2\xe4\x9c\xdco']
Bad pipe message: %s [b")p\x00n\xfe\xac\x9a\xf7\t\x83\xd9'\x9bk\xf5\x06\x17u b\xf0\xde\xc3\x98<\x0e3MN\xc9\x1e\xf3c\x80\x81\x01\xc9C\x01u\xa4\x0b_\xbc\xda\xab\x92\xdc\xf6{\x1e\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff

In [17]:
import numpy as np

In [18]:
(np.array(pred_list) == np.array(ground_truth_list)).mean()

0.7003501750875438

In [19]:
pos_cnt, neg_cnt = 0, 0
for item in dataset_dict['test']:
    y = item['target'][0]
    if y == 1:
        pos_cnt += 1
    else:
        neg_cnt += 1

pos_cnt, neg_cnt

(1000, 999)

In [20]:
len(dataset_dict['train'])

62478